Imports

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
from datetime import datetime
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
import folium
from folium.plugins import HeatMap
import matplotlib.colors as mcolors

Data Preparation

In [11]:
# Load the dataset
file_path = r"C:\Users\poltr\OneDrive - udl.cat\Desktop\MTA_Subway_Hourly_Ridership__2020-2024_20250408.csv"

# Read data from the specified file
df = pd.read_csv(file_path)

# Display the first few rows of the dataframe
df.head()

C:\Users\poltr\AppData\Local\Temp\ipykernel_2968\1957882157.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,Georeference
0,05/31/2021 05:00:00 PM,subway,250,Kings Hwy (F),Brooklyn,metrocard,Metrocard - Students,3,0,40.603218,-73.97236,POINT (-73.97236 40.603218)
1,05/31/2021 09:00:00 AM,subway,318,"34 St-Penn Station (1,2,3)",Manhattan,metrocard,Metrocard - Unlimited 30-Day,43,0,40.750374,-73.99106,POINT (-73.99106 40.750374)
2,05/31/2021 09:00:00 AM,subway,385,176 St (4),Bronx,metrocard,Metrocard - Fair Fare,11,0,40.848480,-73.91180,POINT (-73.9118 40.84848)
3,05/31/2021 08:00:00 PM,subway,198,Howard Beach-JFK Airport (A),Queens,metrocard,Metrocard - Unlimited 7-Day,1,0,40.660477,-73.83030,POINT (-73.8303 40.660477)
4,05/31/2021 09:00:00 PM,subway,2,"Astoria Blvd (N,W)",Queens,metrocard,Metrocard - Unlimited 30-Day,5,0,40.770260,-73.91785,POINT (-73.91785 40.77026)


In [ ]:
df.head()

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,Georeference,date
0,05/31/2021 05:00:00 PM,subway,250,Kings Hwy (F),Brooklyn,metrocard,Metrocard - Students,3,0,40.603218,-73.97236,POINT (-73.97236 40.603218),2021-05-31 17:00:00
1,05/31/2021 09:00:00 AM,subway,318,"34 St-Penn Station (1,2,3)",Manhattan,metrocard,Metrocard - Unlimited 30-Day,43,0,40.750374,-73.99106,POINT (-73.99106 40.750374),2021-05-31 09:00:00
2,05/31/2021 09:00:00 AM,subway,385,176 St (4),Bronx,metrocard,Metrocard - Fair Fare,11,0,40.848480,-73.91180,POINT (-73.9118 40.84848),2021-05-31 09:00:00
3,05/31/2021 08:00:00 PM,subway,198,Howard Beach-JFK Airport (A),Queens,metrocard,Metrocard - Unlimited 7-Day,1,0,40.660477,-73.83030,POINT (-73.8303 40.660477),2021-05-31 20:00:00
4,05/31/2021 09:00:00 PM,subway,2,"Astoria Blvd (N,W)",Queens,metrocard,Metrocard - Unlimited 30-Day,5,0,40.770260,-73.91785,POINT (-73.91785 40.77026),2021-05-31 21:00:00


Data Visualisation Marathon Plots


In [21]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import plotly.io as pio
pio.renderers.default = 'browser'

# Convert timestamp to datetime and extract date/hour
df['transit_timestamp'] = pd.to_datetime(df['transit_timestamp'])
df['date'] = df['transit_timestamp'].dt.date
df['hour'] = df['transit_timestamp'].dt.hour

# Filter for NYC Marathon 2024 (assuming November 3, 2024)
marathon_date = pd.to_datetime('2024-11-03').date()
marathon_day = df[df['date'] == marathon_date]

# Get top stations by ridership (adjust number as needed)
top_stations = marathon_day.groupby('station_complex')['ridership'].sum().nlargest(20).index
marathon_day_top = marathon_day[marathon_day['station_complex'].isin(top_stations)]

# Create time labels for visualization
marathon_day_top['time_label'] = marathon_day_top['transit_timestamp'].dt.strftime('%H:%M')

# Marathon event timeline (adjust times as needed)
marathon_phases = {
    'Start (Wave 1)': '05:00',
    'Start (Wave 2)': '06:00', 
    'Start (Wave 3)': '07:00',
    'Start (Wave 4)': '08:00',
    'Mid-Race': '11:00',
    'Peak Finishers': '13:00',
    'Race Ends': '17:00'
}

## Visualization 1: Interactive Heatmap with Time Slider
# Create aggregated data for heatmap
heatmap_data = marathon_day_top.groupby(['station_complex', 'hour'])['ridership'].sum().reset_index()

# Filter hours to show 4:00-19:00 (4AM-7PM)
heatmap_data = heatmap_data[heatmap_data['hour'].between(4, 19)]

fig1 = px.density_heatmap(
    heatmap_data,
    x='hour',
    y='station_complex',
    z='ridership',
    nbinsx=16,  # For 4AM-7PM (16 hours)
    title='NYC Marathon 2024 - Subway Ridership by Hour (4AM-7PM)',
    labels={'hour': 'Hour of Day', 'station_complex': 'Station', 'ridership': 'Riders'},
    animation_frame='hour',
    range_color=[heatmap_data['ridership'].min(), heatmap_data['ridership'].max()*0.8],
    range_x=[4, 19]  # Explicitly set x-axis range
)

# Add marathon phase annotations outside the main plot
annotations = []
y_pos = 1.15
for phase, time in marathon_phases.items():
    hour = int(time.split(':')[0])
    annotations.append(dict(
        x=hour,
        y=y_pos,
        xref='x',
        yref='paper',
        text=phase,
        showarrow=True,
        arrowhead=1,
        ax=0,
        ay=-25,
        bgcolor='rgba(255,255,255,0.8)',
        font=dict(size=12)
    ))
    y_pos -= 0.08

fig1.update_layout(
    xaxis_title='Hour of Day',
    yaxis_title='Station',
    coloraxis_colorbar_title='Riders',
    height=850,
    xaxis=dict(
        tickvals=list(range(4, 20)),
        ticktext=[f"{h}:00" for h in range(4, 20)],
        range=[4, 19]  # Ensure axis shows full range
    ),
    margin=dict(t=180, b=100, l=100, r=100),
    annotations=annotations,
    title_x=0.5,
    title_y=0.97,
    title_font=dict(size=20)
)

# Adjust animation slider range
fig1.update_layout(
    sliders=[{
        'active': 0,
        'currentvalue': {'prefix': 'Hour: '},
        'steps': [{'args': [[frame.name], {'frame': {'duration': 300, 'redraw': True}}],
                 'label': str(frame.name),
                 'method': 'animate'} for frame in fig1.frames],
        'x': 0.1,
        'len': 0.9,
        'xanchor': 'left',
        'yanchor': 'top',
        'pad': {'t': 50}
    }]
)

# Show the figure
fig1.show()

# Optional: Save to HTML file
# fig1.write_html("marathon_heatmap_final.html")

C:\Users\poltr\AppData\Local\Temp\ipykernel_2968\1841100230.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
## Visualization 2: Final Optimized Line Plot with Perfect Control Placement
# Create pivot table for line plot
pivot_df = marathon_day_top.pivot_table(
    index='transit_timestamp',
    columns='station_complex',
    values='ridership',
    aggfunc='sum'
).resample('H').sum()

# Identify top 5 busiest stations
top_5_stations = marathon_day_top.groupby('station_complex')['ridership'].sum().nlargest(5).index.tolist()

# Create the figure
fig2 = go.Figure()

# Create a color mapping for all stations
color_sequence = px.colors.qualitative.Plotly
station_colors = {station: color_sequence[i % len(color_sequence)] 
                 for i, station in enumerate(pivot_df.columns)}

# Add all station traces with individual colors
for station in pivot_df.columns:
    # Start with highlighted opacity for top stations, faded for others
    initial_opacity = 1.0 if station in top_5_stations else 0.3
    initial_width = 2.5 if station in top_5_stations else 1.5
    
    fig2.add_trace(
        go.Scatter(
            x=pivot_df.index,
            y=pivot_df[station],
            name=station,
            visible=True,
            line=dict(
                color=station_colors[station], 
                width=initial_width,
            ),
            opacity=initial_opacity,
            hovertemplate='<b>%{fullData.name}</b><br>Time: %{x|%H:%M}<br>Riders: %{y:,}<extra></extra>'
        )
    )

# Add marathon phase annotations
annotations = []
y_pos = 1.08  # Start above the plot
for phase, time in marathon_phases.items():
    dt = datetime.strptime(f"2024-11-03 {time}", '%Y-%m-%d %H:%M')
    annotations.append(dict(
        x=dt,
        y=y_pos,
        xref='x',
        yref='paper',
        text=phase,
        showarrow=True,
        arrowhead=1,
        ax=0,
        ay=-30,
        bgcolor='rgba(255,255,255,0.8)',
        font=dict(size=10)
    ))
    y_pos -= 0.07  # Stack annotations vertically
    fig2.add_vline(x=dt, line_dash="dot", line_color="gray", opacity=0.5)

# Add descriptive text above buttons (moved further down)
fig2.add_annotation(
    x=0.5,
    y=-0.25,  # Moved down from -0.18
    xref='paper',
    yref='paper',
    text="Use buttons below to adjust view or click legend items to toggle stations",
    showarrow=False,
    font=dict(size=11, color='gray'),
    align='center'
)

# Update layout with adjusted control positioning
fig2.update_layout(
    title=dict(
        text='NYC Marathon 2024 - Hourly Ridership by Station<br><sup>Top 5 busiest stations highlighted</sup>',
        x=0.5,
        y=0.97,
        xanchor='center',
        font=dict(size=18)
    ),
    xaxis_title='Time of Day (4AM-7PM)',
    yaxis_title='Number of Riders',
    height=900,  # Increased height to 900px for better spacing
    width=1100,
    margin=dict(t=150, b=180, l=100, r=150),  # Increased bottom margin to 180
    plot_bgcolor='rgba(240,240,240,0.8)',
    annotations=annotations,
    hovermode='x unified',
    xaxis=dict(
        tickformat='%H:%M',
        range=[datetime(2024, 11, 3, 4, 0), datetime(2024, 11, 3, 19, 0)],
        rangeslider=dict(
            visible=True,
            thickness=0.05,
            range=[datetime(2024, 11, 3, 4, 0), datetime(2024, 11, 3, 19, 0)]
        ),
        type='date'
    ),
    legend=dict(
        orientation='v',
        yanchor='middle',
        y=0.5,
        xanchor='left',
        x=1.02,
        itemclick='toggle',
        itemdoubleclick='toggleothers',
        font=dict(size=10),
        traceorder='normal'
    )
)

# Add control buttons moved further down
fig2.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(
                    label="⏮ Reset All",
                    method="restyle",
                    args=[{"opacity": [0.3]*len(pivot_df.columns)},
                          {"line.width": [1.5]*len(pivot_df.columns)}],
                    args2=[None]
                ),
                dict(
                    label="⭐ Highlight Top 5",
                    method="restyle",
                    args=[{"opacity": [1.0 if station in top_5_stations else 0.3 for station in pivot_df.columns]},
                          {"line.width": [2.5 if station in top_5_stations else 1.5 for station in pivot_df.columns]}]
                ),
                dict(
                    label="⏭ Show All",
                    method="restyle",
                    args=[{"opacity": [1.0]*len(pivot_df.columns)},
                          {"line.width": [2.0]*len(pivot_df.columns)}]
                )
            ],
            direction='right',
            x=0.5,
            xanchor='center',
            y=-0.30,  # Moved down from -0.22
            yanchor='top',
            bgcolor='rgba(240,240,240,0.9)',
            bordercolor='rgba(100,100,100,0.3)',
            borderwidth=1,
            font=dict(size=12)
        )
    ]
)

fig2.show()

C:\Users\poltr\AppData\Local\Temp\ipykernel_2968\2987126294.py:8: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



KeyError: 'station_complex'

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta

# Filter for December 31, 2023 and January 1, 2024
nye_data = df[(df['date'] == pd.to_datetime('2023-12-31').date()) | 
             (df['date'] == pd.to_datetime('2024-01-01').date())]

# Aggregate ridership by hour across all stations
nye_hourly = nye_data.groupby(['date', 'hour'])['ridership'].sum().reset_index()

# Create the figure
fig = go.Figure()

# Add NYE trace
fig.add_trace(go.Scatter(
    x=nye_hourly[nye_hourly['date'] == pd.to_datetime('2023-12-31').date()]['hour'],
    y=nye_hourly[nye_hourly['date'] == pd.to_datetime('2023-12-31').date()]['ridership'],
    name='New Year\'s Eve (Dec 31)',
    line=dict(color='gold', width=3),
    hovertemplate='<b>NYE</b><br>Hour: %{x}:00<br>Riders: %{y:,}<extra></extra>'
))

# Add New Year's Day trace
fig.add_trace(go.Scatter(
    x=nye_hourly[nye_hourly['date'] == pd.to_datetime('2024-01-01').date()]['hour'],
    y=nye_hourly[nye_hourly['date'] == pd.to_datetime('2024-01-01').date()]['ridership'],
    name='New Year\'s Day (Jan 1)',
    line=dict(color='royalblue', width=3),
    hovertemplate='<b>New Year\'s Day</b><br>Hour: %{x}:00<br>Riders: %{y:,}<extra></extra>'
))

# Add midnight marker
fig.add_vline(
    x=0,  # Midnight (hour 0)
    line_dash="dash",
    line_color="red",
    annotation_text="Midnight",
    annotation_position="top left"
)

# Update layout
fig.update_layout(
    title='New Year\'s Eve vs New Year\'s Day Subway Ridership<br><sub>Comparison of hourly ridership patterns</sub>',
    xaxis_title='Hour of Day',
    yaxis_title='Total Ridership',
    height=600,
    width=900,
    plot_bgcolor='rgba(240,240,240,0.8)',
    hovermode='x unified',
    xaxis=dict(
        tickvals=list(range(24)),
        ticktext=[f"{h}:00" for h in range(24)],
        range=[-0.5, 23.5]  # Show full 24-hour cycle
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    annotations=[
        dict(
            x=0.5,
            y=-0.2,
            xref='paper',
            yref='paper',
            text="Note: Midnight (0:00) marks the transition from NYE to New Year's Day",
            showarrow=False,
            font=dict(size=10, color='gray')
        )
    ]
)

# Add range slider to focus on specific hours
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True,
            thickness=0.05
        ),
        type='linear'
    )
)

fig.show()

In [65]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

# First, let's check what dates are actually available in your data
def get_available_dates(file_path, chunk_size=100000):
    dates = set()
    for chunk in pd.read_csv(file_path, chunksize=chunk_size, usecols=['transit_timestamp']):
        chunk_dates = pd.to_datetime(chunk['transit_timestamp']).dt.date
        dates.update(chunk_dates)
    return sorted(dates)

file_path = r"C:\Users\poltr\OneDrive - udl.cat\Desktop\MTA_Subway_Hourly_Ridership__2020-2024_20250408.csv"
available_dates = get_available_dates(file_path)

# Find the most recent New Year's period in the data
nye_dates = []
for year in range(2023, 2019, -1):  # Check backwards from 2023 to 2020
    nye = datetime(year, 12, 31).date()
    jan1 = datetime(year+1, 1, 1).date()
    if nye in available_dates and jan1 in available_dates:
        nye_dates = [nye, jan1]
        break

if not nye_dates:
    raise ValueError("No complete New Year's Eve/New Year's Day data found in the dataset")

print(f"Using data for NYE: {nye_dates[0]}, New Year's Day: {nye_dates[1]}")

# Define the stations we're interested in
times_square_stations = [
    '42 St-Port Authority',
    'Times Sq-42 St',
    '34 St-Penn Station',
    '59 St-Columbus Circle'
]

# Process the data in chunks
nye_data = pd.DataFrame()
for chunk in pd.read_csv(
    file_path,
    chunksize=100000,
    usecols=['transit_timestamp', 'station_complex', 'ridership'],
    parse_dates=['transit_timestamp']
):
    chunk['date'] = chunk['transit_timestamp'].dt.date
    filtered_chunk = chunk[
        (chunk['date'].isin(nye_dates)) & 
        (chunk['station_complex'].isin(times_square_stations))
    ]
    nye_data = pd.concat([nye_data, filtered_chunk])

# Extract hour from timestamp
nye_data['hour'] = nye_data['transit_timestamp'].dt.hour

# Aggregate by station, date, and hour
station_hourly = nye_data.groupby(['station_complex', 'date', 'hour'])['ridership'].sum().unstack(level=0)

# Create the visualization
fig = go.Figure()

# Color palette
colors = px.colors.qualitative.Plotly

# Add each station's NYE pattern
for i, station in enumerate(times_square_stations):
    fig.add_trace(go.Scatter(
        x=station_hourly.loc[nye_dates[0]].index,
        y=station_hourly.loc[nye_dates[0]][station],
        name=f'{station} (NYE)',
        line=dict(color=colors[i], width=3),
        hovertemplate=f'<b>{station}</b><br>Hour: %{{x}}:00<br>Riders: %{{y:,}}<extra></extra>'
    ))

# Add each station's New Year's Day pattern
for i, station in enumerate(times_square_stations):
    fig.add_trace(go.Scatter(
        x=station_hourly.loc[nye_dates[1]].index,
        y=station_hourly.loc[nye_dates[1]][station],
        name=f'{station} (Jan 1)',
        line=dict(color=colors[i], width=3, dash='dot'),
        hovertemplate=f'<b>{station}</b><br>Hour: %{{x}}:00<br>Riders: %{{y:,}}<extra></extra>',
        visible=False
    ))

# Add interactive controls
buttons = [
    dict(label="NYE Only", method="update",
         args=[{"visible": [True]*4 + [False]*4},
               {"title": f"New Year's Eve {nye_dates[0].year} Ridership"}]),
    dict(label="Compare", method="update",
         args=[{"visible": [True]*8},
               {"title": f"NYE vs New Year's Day {nye_dates[0].year}"}]),
    dict(label="Jan 1 Only", method="update",
         args=[{"visible": [False]*4 + [True]*4},
               {"title": f"New Year's Day {nye_dates[1].year} Recovery"}])
]

fig.update_layout(
    title=f"Times Square Stations: NYE {nye_dates[0].year} vs New Year's Day",
    xaxis_title="Hour of Day",
    yaxis_title="Total Ridership",
    height=700,
    width=1000,
    plot_bgcolor='rgba(240,240,240,0.8)',
    hovermode='x unified',
    xaxis=dict(
        tickvals=list(range(24)),
        ticktext=[f"{h}:00" for h in range(24)],
        range=[16, 6]  # Focus on evening through morning
    ),
    updatemenus=[dict(
        type="buttons",
        direction="right",
        x=0.5, xanchor="center",
        y=1.15, yanchor="top",
        buttons=buttons
    )]
)

fig.show()

C:\Users\poltr\AppData\Local\Temp\ipykernel_2968\3541503965.py:9: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\poltr\AppData\Local\Temp\ipykernel_2968\3541503965.py:9: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\poltr\AppData\Local\Temp\ipykernel_2968\3541503965.py:9: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\poltr\AppData\Local\Temp\ipykernel_2968\3541503965.py:9: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a f

Using data for NYE: 2023-12-31, New Year's Day: 2024-01-01


C:\Users\poltr\AppData\Local\Temp\ipykernel_2968\3541503965.py:40: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\poltr\AppData\Local\Temp\ipykernel_2968\3541503965.py:40: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\poltr\AppData\Local\Temp\ipykernel_2968\3541503965.py:40: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\poltr\AppData\Local\Temp\ipykernel_2968\3541503965.py:40: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify

KeyError: datetime.date(2023, 12, 31)